In [1]:
import os
from os.path import join as oj
import sys, time
sys.path.insert(1, oj(sys.path[0], '..'))  # insert parent path
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from copy import deepcopy
import pickle as pkl
import pandas as pd
sys.path.append('../models')
sys.path.append('../fit')

from model import LSTMSentiment
from torchtext import data
from torchtext import datasets
import torch
import cd
import pandas as pd

%matplotlib inline
%load_ext autoreload
%autoreload 2

# load dset + model

In [2]:
# data params
vector_cache =  '../data/.vector_cache/input_vectors.pt'
word_vectors ='glove.6B.300d'
batch_size = 50
device = 'cuda' if torch.cuda.is_available() else 'cpu'


# load dset
inputs = data.Field(lower= True)
answers = data.Field(sequential=False, unk_token=None)
train, dev, test = datasets.SST.splits(inputs, answers, fine_grained=False, train_subtrees=True,
                                       filter_pred=lambda ex: ex.label != 'neutral')
inputs.build_vocab(train, dev, test)

if os.path.isfile(vector_cache):
    inputs.vocab.vectors = torch.load(vector_cache)
else:
    inputs.vocab.load_vectors(word_vectors)
    os.makedirs(os.path.dirname(vector_cache), exist_ok=True)
    torch.save(inputs.vocab.vectors, vector_cache)
answers.build_vocab(train)

train_iter, dev_iter, test_iter = data.BucketIterator.splits(
    (train, dev, test), batch_size=batch_size, device=torch.device(0))

# load model
model_path = "../models/init_models"
model_list = os.listdir(model_path)
model1 = torch.load(os.path.join(model_path, model_list[0]), map_location=torch.device(0)).eval()

model2 = torch.load(os.path.join(model_path, model_list[1]), map_location=torch.device(0)).eval()

/home/lauri/.conda/envs/gpu_usage/lib/python3.6/site-packages/torch/serialization.py:435: SourceChangeWarning: source code of class 'model.LSTMSentiment' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


# evaluate cd word-level

In [ ]:
inputs.vocab.words

In [72]:
# choose hyperparams
it = test_iter
m = model1

# what to store
words = {}

it.init_epoch()
# check out how two models differ
import torch.optim as O
import torch.nn as nn
criterion = nn.CrossEntropyLoss()
n_dev_correct, dev_loss = 0, 0

# remember batches are num_words x batch_size
for batch in tqdm(it):
    answer1 = m(batch)
    num_words = batch.text.shape[0]
    batch_size = batch.text.shape[1]
    
    for word_num in range(num_words):
        word_per_batch = batch.text[word_num] # gets word at same place for all batches
        
        # get cd scores for each word
        rel, _ = cd.cd_batch_text(batch, m, start=word_num, stop=word_num + 1)
        rel = rel.softmax(dim=0) # 2 x batch_size
        rel = rel[0] # only get positive class
        
        # actually get the words
        for batch_num in range(word_per_batch.shape[0]):
            word = inputs.vocab.itos[word_per_batch[batch_num]]
            score = rel[batch_num].item()
            
            # add to store
            if not word in words:
                words[word] = (1, score) # count, sum
            else:
                (count, running_sum) = words[word]
                words[word] = (count + 1, running_sum + score)
#     n_dev_correct += (((torch.max(answer2, 1)[1].view(dev_batch.label.size()).data == dev_batch.label.data))).sum()
#     dev_loss = criterion(answer2, dev_batch.label)
# dev_acc = 100. * n_dev_correct / len(dev)
# print(dev_acc.item(), dev_loss.item())

100%|██████████| 37/37 [01:18<00:00,  7.17s/it]


# look at fairness results

In [112]:
results = pd.DataFrame()
results['word'] = words.keys()
results['count'] = [words[word][0] for word in results['word']]
results['sent'] = [words[word][1] / words[word][0] for word in results['word']]
results = results.sort_values(by=['sent'], ascending=False)
results.to_pickle('results/word_fairness_test.pkl')

In [117]:
r = pd.read_pickle('results/word_fairness_test.pkl')
r[r['count'] >= 5]

,word,count,sent
31,brilliant,5,0.814058
55,wonderful,7,0.792790
550,mesmerizing,5,0.791960
125,enjoy,11,0.786107
1542,honesty,5,0.783679
1311,pleasure,8,0.781759
210,intelligent,13,0.768969
3708,creative,5,0.766074
10,excellent,7,0.765148
796,polished,6,0.765092


In [101]:
comparisons = [('actor', 'actress'), ('black', 'white'), 
               ('him', 'her'), ('young', 'old'), ('latino', 'asian'),
               ('romance', 'comedy')]
for (x, y) in comparisons:
    print(r[r.word == x], '\n', r[r.word == y], '\n')

      word  count      sent
962  actor     12  0.555093 
        word  count      sent
68  actress      4  0.541827 

      word  count      sent
553  black      8  0.461766 
        word  count     sent
5584  white      2  0.52095 

    word  count      sent
915  him      5  0.493404 
     word  count      sent
292  her     38  0.614049 

       word  count      sent
1840  young      8  0.602673 
     word  count      sent
618  old     16  0.430794 

        word  count      sent
5783  latino      1  0.609252 
        word  count      sent
1794  asian      2  0.637687 

        word  count      sent
920  romance     11  0.650436 
        word  count      sent
116  comedy     62  0.614354 

